# Summary
Here, we combine all impact layers to create a composite human impact layers.

# Setting up the environment

In [ ]:
# Import and/or install libraries
import subprocess, os, gcsfs, json

try:
    import geemap, ee
except ImportError:
    subprocess.check_call(["python", '-m', 'pip', 'install', '-U', 'geemap'])
    import geemap, ee


In [ ]:
# Connect to Google Drive to access files

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Connect to Google Earth Engine if neccessary

service_account = os.environ.get('GOOGLE_SERVICE_ACCOUNT')
credentials = ee.ServiceAccountCredentials(service_account, os.environ.get('GOOGLE_APPLICATION_CREDENTIALS'))
ee.Initialize(credentials)

# Import layers

In [ ]:
from google.cloud import storage
def list_blobs(bucket_name, prefix):
    storage_client = storage.Client.from_service_account_json(os.environ.get('GOOGLE_APPLICATION_CREDENTIALS'))

    blobs = storage_client.list_blobs(bucket_name, prefix=prefix)
    blob_names = []

    for blob in blobs:
      if blob.name != prefix:
        blob_names.append('gs://' + bucket_name + '/' + blob.name)

    return blob_names

built_blobs = list_blobs('nature-watch-bucket', 'COGS/built/2023/')
built_images = geemap.load_GeoTIFFs(built_blobs)
built_projection =  ee.Image(built_images.first()).projection()
built = built_images.mosaic().setDefaultProjection(built_projection)

crop_blobs = list_blobs('nature-watch-bucket', 'COGS/crop/2019/')
crop_images = geemap.load_GeoTIFFs(crop_blobs)
crop_projection =  ee.Image(crop_images.first()).projection()
crop = crop_images.mosaic().setDefaultProjection(crop_projection)

fire_blobs = list_blobs('nature-watch-bucket', 'COGS/fire/2022/')
fire_images = geemap.load_GeoTIFFs(fire_blobs)
fire_projection =  ee.Image(fire_images.first()).projection()
fire = fire_images.mosaic().setDefaultProjection(fire_projection)

mines_features = ee.FeatureCollection("projects/sat-io/open-datasets/global-mining/global_mining_polygons")
mines = mines_features.reduceToImage(
  properties=['AREA'],
  reducer=ee.Reducer.first()
).gt(0).selfMask().select(['first']).unmask(0)

treeloss = ee.Image("UMD/hansen/global_forest_change_2022_v1_10").select('lossyear').unmask(0).gt(0).selfMask()


# Reproject to 1km2 pixels

In [ ]:
project_proj = built.projection()

built_resampled = built.reduceResolution(
      reducer = ee.Reducer.sum().unweighted(),
      maxPixels=1200 # slightly more than 1111
      ).reproject(project_proj, None, 1000)

crop_resampled = crop.reduceResolution(
      reducer = ee.Reducer.sum().unweighted(),
      maxPixels=1200 # slightly more than 1111
      ).reproject(project_proj, None, 1000)

treeloss_resampled = treeloss.reduceResolution(
      reducer = ee.Reducer.sum().unweighted(),
      maxPixels=1400 # slightly more than 1045
      ).reproject(project_proj, None, 1000)

fire_resampled = fire.reduceResolution(
      reducer = ee.Reducer.max(),
      maxPixels=1400 # slightly more than 1045
      ).reproject(project_proj, None, 1000)

mines_resampled = mines.setDefaultProjection(built_projection).reduceResolution(
      reducer = ee.Reducer.sum().unweighted(),
      maxPixels=1200 # slightly more than 1045
      ).reproject(project_proj, None, 1000)



In [ ]:
print(mines_resampled.getInfo())

# Create composite

## Normalise the layers

In [ ]:
## Filter out noise
built_selected = built_resampled.updateMask(built_resampled.gt(50)).unmask(0)
crop_selected = crop_resampled.updateMask(crop_resampled.gt(50)).unmask(0)
treeloss_selected = treeloss_resampled.updateMask(treeloss_resampled.gt(100)).unmask(0)
fire_selected = fire_resampled.updateMask(fire_resampled.gt(8)).unmask(0)
mines_selected = mines_resampled.updateMask(mines_resampled.gt(50)).unmask(0)

In [ ]:
# Normalize an image with known min and max.
def normalize_image(image, min_value, max_value):
    return image.subtract(min_value).divide(max_value - min_value)

# Apply the normalization function to each resampled image.
built_normalized = normalize_image(built_selected, 49, 1111)
crop_normalized = normalize_image(crop_selected, 49, 1111)
treeloss_normalized = normalize_image(treeloss_selected, 99, 1045)
fire_normalized = normalize_image(fire_selected, 7, 26)
mines_normalized = normalize_image(mines_selected, 49, 1111)


In [ ]:
## Create composite
# Define the weights
built_weight = 1
crop_weight = 0.5
treeloss_weight = 0.5
intense_fire_weight = 1
mine_weight = 1


# Multiply each image by its weight
weighted_built = built_normalized.multiply(built_weight)
weighted_crop = crop_normalized.multiply(crop_weight)
weighted_treeloss = treeloss_normalized.multiply(treeloss_weight)
weighted_fire = fire_normalized.multiply(intense_fire_weight)
weighted_mine = mines_normalized.multiply(mine_weight)

# Create an ImageCollection from the weighted images
weighted_collection = ee.ImageCollection([
    weighted_built.rename('value'),
    weighted_crop.rename('value'),
    weighted_treeloss.rename('value'),
    weighted_fire.rename('value'),
    weighted_mine.rename('value')
])

# Take the maximum value for each pixel across the images
max_value_image = weighted_collection.max()


In [ ]:
print(max_value_image.bandNames().getInfo())

# Export

In [ ]:
block_list = list(range(1, 26))

blocks = []
with open("/content/drive/MyDrive/data/blocks.geojson") as f:
    json_data = json.load(f)
    for block_id in block_list:
        for feature in json_data['features']:
            if feature['properties']['id'] == block_id:
                feature['properties']['style'] = {}
                blocks.append(feature)

ee_blocks = [geemap.geojson_to_ee(block) for block in blocks]

In [ ]:
# Loop through blocks and export
for id, block in enumerate(ee_blocks):
  if id != 19:
    image_name = 'all2023_' + str(id)
    fileNamePrefix = 'COGS/all/2023/' + image_name


    exportConfig = {
        'image': max_value_image,
        'description': image_name,
        'bucket': 'nature-watch-bucket',
        'fileNamePrefix': fileNamePrefix,
        'scale': 1000,
        'maxPixels': 3867395000,
        'region': block,
        'fileFormat': 'GeoTIFF',
        'formatOptions': {'cloudOptimized': True}
    }

    task = ee.batch.Export.image.toCloudStorage(**exportConfig)
    task.start()

In [ ]:
# Or just export one block
id = 19
block = ee_blocks[id]


image_name = 'all2023_' + str(id)
fileNamePrefix = 'COGS/all/2023/' + image_name


exportConfig = {
    'image': max_value_image,
    'description': image_name,
    'bucket': 'nature-watch-bucket',
    'fileNamePrefix': fileNamePrefix,
    'scale': 1000,
    'maxPixels': 3867395000,
    'region': block,
    'fileFormat': 'GeoTIFF',
    'formatOptions': {'cloudOptimized': True}
}

task = ee.batch.Export.image.toCloudStorage(**exportConfig)
task.start()

In [ ]:
ee.data.listOperations()

In [ ]:
Map = geemap.Map()
Map.add_basemap('SATELLITE')
Map.addLayer(fire_selected.updateMask(fire_selected.gt(0)), {'min':0, 'max': 26, 'palette': ['white', 'red']}, 'fire')
Map.addLayer(crop_selected.updateMask(crop_selected.gt(0)), {'min':0, 'max': 1111, 'palette': ['white', 'orange']}, 'crop')
Map.addLayer(built_selected.updateMask(built_selected.gt(0)), {'min':0, 'max': 1111, 'palette': ['white', 'black']}, 'built')
Map.addLayer(treeloss_selected.updateMask(treeloss_selected.gt(0)), {'min':0, 'max': 1045, 'palette': ['white', 'pink']}, 'treeloss')

Map.addLayer(max_value_image.updateMask(max_value_image.gt(0)), {'min':0, 'max': 1, 'palette': ['pink', 'black']}, 'composite')

# Map.addLayer(mines_resampled, {'min':0, 'max': 1, 'palette': ['white', 'black']}, 'mines_resampled')
# Map.addLayer(mines, {'min':0, 'max': 1, 'palette': ['white', 'black']}, 'mines')

Map.setCenter(14.80, 1.15, 10)
Map